In [5]:
import pandas as pd

%load_ext autoreload
%autoreload 2

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper'.format(folder)
bed_folder = '{}/bed'.format(folder)
# folder2 = '/projects/ps-yeolab2/obotvinnik/singlecell_pnm'

splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)


import pybedtools

alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive = pybedtools.BedTool(constitutive_bedfile)

from Bio import SeqIO
import kvector
import pybedtools
import pyhomer

DIRECTIONS = 'upstream', 'downstream'

placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
conservation_bed = pybedtools.BedTool(placental_filename)

genome = 'hg19'

nt = 200

genome_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

pair = pyhomer.ForegroundBackgroundPair(alt_exons, constitutive)


/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [6]:
alternative_feature_folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative'

In [7]:
constitutive_feature_folder

'/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive'

In [16]:
ls $alternative_feature_folder/*.bed

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exon2_phastcons_placental_mammal.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_downstream200nt.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_merkin2012_ancient_alt_exons.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_phastcons_placental_mammal.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_stop_plus1.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_foreground.bed
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200_placental_f

In [17]:
! head $alt_exons_bedfile > $alternative_feature_folder/test.bed

Cacluate the time needed for 10 intervals

In [18]:
%%time

import kvector

kmers = kvector.kmer.per_interval_kmers('{}/test.bed'.format(alternative_feature_folder), genome_fasta, conservation_bed)

Process PoolWorker-14:
Process PoolWorker-11:
Process PoolWorker-16:
Process PoolWorker-15:
Process PoolWorker-13:
Process PoolWorker-12:
Process PoolWorker-5:
Process PoolWorker-1:
Process PoolWorker-2:
Process PoolWorker-10:
Process PoolWorker-6:
Process PoolWorker-4:
Process PoolWorker-3:
Process PoolWorker-9:
Process PoolWorker-8:
Process PoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/obotvinnik/anaconda/lib/python2.7/multiprocessing/process.py", line

KeyboardInterrupt: 

```
CPU times: user 1.52 s, sys: 103 ms, total: 1.62 s
Wall time: 3min 32s
```

wow, rounded 4 min per 10 intervals! that's so much! so

In [30]:
%%file ~/count_interval_kmers.py

import sys
import kvector

bedfile = sys.argv[1]
genome_fasta = sys.argv[2]
other = sys.argv[3]
threads = int(sys.argv[4])

kmers = kvector.kmer.per_interval_kmers(bedfile, genome_fasta, other, threads=threads)
kmers = kmers.astype(int)
csv = bedfile.replace('.bed', '_kmers.csv')
kmers.to_csv(csv)

Overwriting /home/obotvinnik/count_interval_kmers.py


In [31]:
n_lines = ! wc -l $alt_exons_bedfile
n_lines = int(n_lines[0].split()[0])/10
n_lines

4390

In [32]:
# ! python ~/count_interval_kmers.py \
#     /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons.bed \
#     /projects/ps-yeolab/genomes/hg19/chromosomes/all.fa /projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed

In [33]:
import qtools

In [39]:
def kmer_command(bedfile, genome_fasta, other, threads):
    return 'python ~/count_interval_kmers.py {} {} {} {}'.format(bedfile, genome_fasta, other, threads)

bedfiles = alt_exons_bedfile, #constitutive_bedfile
threads = 8

for bedfile in bedfiles:
    jobname = bedfile.split('/')[-2] + '_kmers'
    
    commands = []
    
    n_lines = ! wc -l $bedfile
    n_lines = int(n_lines[0].split()[0])
    n_minutes = 4 * n_lines / 10
    
    commands.append(kmer_command(bedfile, genome_fasta, placental_filename, threads))
    
    for direction in DIRECTIONS:
        bed = pybedtools.BedTool(bedfile)
        if direction == 'downstream':
            intron = bed.flank(l=0, r=nt, s=True, genome=genome)
        elif direction == 'upstream':
            intron = bed.flank(l=nt, r=0, s=True, genome=genome)
        intron_filename = bedfile.replace('.bed', '_{}{}nt.bed'.format(direction, nt))
        intron.saveas(intron_filename)
        commands.append(kmer_command(intron_filename, genome_fasta, placental_filename, threads))
#     print commands
    qtools.Submitter(commands, jobname, walltime='48:00:00'.format(n_minutes), array=True, ppn=threads)

running 3 tasks as an array-job.
Wrote commands to alternative_kmers.sh.
Submitted script to queue home.
 Job ID: 5097012


In [1]:
ls *kmers*

0.2.25_intron_kmers.ipynb               constitutive_kmers.sh.err-1
0.2.26_submit_job_to_count_kmers.ipynb  constitutive_kmers.sh.err-2
alternative_kmers.sh                    constitutive_kmers.sh.err-3
alternative_kmers.sh.err-1              constitutive_kmers.sh.out-1
alternative_kmers.sh.err-2              constitutive_kmers.sh.out-2
alternative_kmers.sh.err-3              constitutive_kmers.sh.out-3
alternative_kmers.sh.out-1              count_kmers.sh
alternative_kmers.sh.out-2              count_kmers.sh.err
alternative_kmers.sh.out-3              count_kmers.sh.out
constitutive_kmers.sh


In [2]:
! tail *kmers*.out*

==> alternative_kmers.sh.out-1 <==
Nodes:        tscc-2-51

==> alternative_kmers.sh.out-2 <==
Nodes:        tscc-2-49

==> alternative_kmers.sh.out-3 <==
Nodes:        tscc-2-50

==> constitutive_kmers.sh.out-1 <==
Nodes:        tscc-2-49

==> constitutive_kmers.sh.out-2 <==
Nodes:        tscc-2-50

==> constitutive_kmers.sh.out-3 <==
Nodes:        tscc-2-51

==> count_kmers.sh.out <==
Nodes:        tscc-2-49


In [8]:
! ls $alternative_feature_folder/*kmers*

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_downstream200nt_kmers.csv
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_kmers.csv
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/exons_upstream200nt_kmers.csv


In [9]:
! ls $constitutive_feature_folder/*kmers*

/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/exons_downstream200nt_kmers.csv
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/exons_kmers.csv
/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/exons_upstream200nt_kmers.csv


In [3]:
! tail *kmers*.err*

==> alternative_kmers.sh.err-1 <==
/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())

==> alternative_kmers.sh.err-2 <==
/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())

==> alternative_kmers.sh.err-3 <==
/